In [35]:
from match_functions import get_match_database
data = get_match_database('2025-03-01', '2025-03-15', leagues = ["Ligue 1"], save = True, add = False)

Extraction des liens...



100%|██████████| 15/15 [02:05<00:00,  8.38s/it]


Chargement de la base de données...
 
Extraction terminée en  4.52 minutes


In [38]:
import ace_tools_open as tools
tools.display_dataframe_to_user("Data", data)

Data


# Links Whoscored

In [ ]:
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from datetime import datetime
from tqdm import tqdm
import time


def get_link(soup):
    code = soup.find_all("a")
    for c in code:
        if "Centre du Match" in c.text:
            if c.get("href") != "#":
                link = c.get("href")
            else:
                link = "None"
            break

    return link


def whoscored_link_traitement(df):
    pattern = r"/matches/(?P<num_lien>\d+)/live/(?P<pays>[^-]+)-(?P<ligue>[^-]+-\S+)-(?P<annee>\d{4}-\d{4})-(?P<equipes>.+)"

    df_extracted = df["link"].str.extract(pattern)
    df_extracted = df.join(df_extracted)
    df_extracted['num_lien'] = df_extracted['link'].str.split("/").str[2]
    df_extracted["num_lien"] = pd.to_numeric(df_extracted["num_lien"], errors="coerce")
    return df_extracted


def whoscored_not_available_error_traitement(df):
    df['num_lien'] = df['link'].str.split("/").str[4]
    df["num_lien"] = pd.to_numeric(df["num_lien"], errors="coerce")
    return df


def get_link_whoscored(range_down, range_up, new):

    path = "Whoscored/urls/raw/"

    number_list = list(range(range_down, range_up))
    pre_links = []
    for el in number_list:
        pre_links.append("https://fr.whoscored.com/matches/" + f"{el}")

    if new == False:
        all_matches = pd.read_csv(f"{path}links.csv")
        try:
            all_errors = pd.read_csv(f"{path}errors.csv")
        except:
            all_errors = pd.DataFrame()
        all_not_availables = pd.read_csv(f"{path}not_availables.csv")

    else:
        all_matches = pd.DataFrame()
        all_not_availables = pd.DataFrame()
        all_errors = pd.DataFrame()

    links = []
    errors = []
    not_availables = []

    i = 0
    j = 0
    k = 0

    options = Options()
    options.add_argument("--headless")  # Mode headless activé
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")

    service = Service(GeckoDriverManager().install())
    driver = webdriver.Firefox(service=service, options=options)

    try:
        for pre_link in tqdm(pre_links):
            try:
                driver.get(pre_link)
                html = driver.page_source
                soup = BeautifulSoup(html, "html.parser")
                link = get_link(soup)
                if link == "None":
                    not_availables.append(pre_link)
                    j+=1
                else:
                    links.append(link)
                    i+=1

                if i % 10 == 0:
                    # Link
                    matches = pd.DataFrame(links, columns=["link"])
                    all_matches = pd.concat([all_matches, matches])
                    all_matches.to_csv(f"{path}links.csv", index=False)
                    i = 0
                    links = []

                if j % 10 == 0:
                    # not_available
                    not_availables = pd.DataFrame(not_availables, columns=["link"])
                    all_not_availables = pd.concat([all_not_availables, not_availables])
                    all_not_availables.to_csv(f"{path}not_availables.csv", index=False)
                    j=0
                    not_availables = []

            except Exception as e:
                print(f"❌ Erreur lors de la récupération de la page : {e}")
                errors.append(pre_link)
                k += 1
            
            if k % 1 == 0:
                #error
                errors = pd.DataFrame(errors, columns=["link"])
                all_errors = pd.concat([all_errors, errors])
                all_errors.to_csv(f"{path}errors.csv", index=False)
                j=0
                errors = []
            
        matches = pd.DataFrame(links, columns=["link"])
        not_availables = pd.DataFrame(not_availables, columns=["link"])
        errors = pd.DataFrame(errors, columns=["link"])

        all_matches = pd.concat([all_matches, matches])
        all_errors = pd.concat([all_errors, errors])
        all_not_availables = pd.concat([all_not_availables, not_availables])

        all_matches.to_csv(f"{path}links.csv", index=False)
        all_errors.to_csv(f"{path}errors.csv", index=False)
        all_not_availables.to_csv(f"{path}not_availables.csv", index=False)

    finally:
        driver.quit()

    # Clean part
    all_matches = whoscored_link_traitement(pd.read_csv(f"{path}links.csv"))
    all_not_availables = whoscored_not_available_error_traitement(pd.read_csv(f"{path}not_availables.csv"))
    all_errors = whoscored_not_available_error_traitement(pd.read_csv(f"{path}errors.csv"))

    all_matches.to_csv("Whoscored/urls/clean/links.csv", index=False)
    all_errors.to_csv("Whoscored/urls/clean/errors.csv", index=False)
    all_not_availables.to_csv("Whoscored/urls/clean/not_availables.csv", index=False)


In [ ]:
get_link_whoscored(1820000, 1830001, True)

 10%|▉         | 996/10001 [21:10<24:31:06,  9.80s/it]

❌ Erreur lors de la récupération de la page : cannot access local variable 'link' where it is not associated with a value


 10%|▉         | 1000/10001 [21:26<12:31:14,  5.01s/it]

❌ Erreur lors de la récupération de la page : cannot access local variable 'link' where it is not associated with a value
❌ Erreur lors de la récupération de la page : cannot access local variable 'link' where it is not associated with a value
❌ Erreur lors de la récupération de la page : cannot access local variable 'link' where it is not associated with a value
❌ Erreur lors de la récupération de la page : cannot access local variable 'link' where it is not associated with a value
❌ Erreur lors de la récupération de la page : cannot access local variable 'link' where it is not associated with a value


 10%|█         | 1006/10001 [21:26<4:18:55,  1.73s/it] 

...

 19%|█▊        | 1869/10001 [28:21<18:37:27,  8.24s/it]

❌ Erreur lors de la récupération de la page : cannot access local variable 'link' where it is not associated with a value


 19%|█▊        | 1870/10001 [28:51<27:26:39, 12.15s/it]

❌ Erreur lors de la récupération de la page : cannot access local variable 'link' where it is not associated with a value


 38%|███▊      | 3791/10001 [1:10:49<1:56:01,  1.12s/it]


In [ ]:
import ace_tools_open as tools
import pandas as pd

links = pd.read_csv("Whoscored/clean/links.csv")
not_available = pd.read_csv("Whoscored/clean/not_availables.csv")
try:
    error = pd.read_csv("Whoscored/clean/errors.csv")
except:
    error = pd.DataFrame()

print("Intervalle des données :", int(links["num_lien"].min()), max([int(links["num_lien"].max()), int(not_available["num_lien"].max())]))
print("Maximum :", max([int(links["num_lien"].max()), int(not_available["num_lien"].max())]))
print("Nombre d'observation :", len(links) + len(not_available))
print("\n")
tools.display_dataframe_to_user("Links :", links)
tools.display_dataframe_to_user("Not availables :", not_available)
tools.display_dataframe_to_user("Errors :", error)
tools.display_dataframe_to_user("NA's :", links[links["num_lien"].isna()])

Intervalle des données : 1820000 1830000
Maximum : 1830000
Nombre d'observation : 9986


Links :


Not availables :


Errors :


NA's :


In [ ]:
df_france = links[links["pays"] == "france"]
df_ligue1 = df_france[df_france["ligue"] == "ligue-1"]
club = df_ligue1[df_ligue1["equipes"].str.contains("montpellier", case=False, na=False)]
import ace_tools_open as tools
tools.display_dataframe_to_user("", club)

# Current month link Whoscored

In [164]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.firefox import GeckoDriverManager
from bs4 import BeautifulSoup
import time

def get_link_current_month(url):

    options = Options()
    options.add_argument("--headless")  # Mode headless activé
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")

    service = Service(GeckoDriverManager().install())
    driver = webdriver.Firefox(service=service, options=options)

    all_links = []

    try:
        driver.get(url)
        
        # Accepter les cookies si la bannière apparaît
        try:
            accept_cookies = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "qc-cmp-cleanslate"))
            )
            driver.execute_script("arguments[0].click();", accept_cookies)
            time.sleep(2)
        except Exception:
            print("Bannière de cookies non détectée.")
        
        time.sleep(2)  # Attendre le chargement de la page
        
        # Récupération du HTML
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        
        # Extraction des liens
        code = soup.find_all("div", class_="Match-module_right_oddsOn__o-ux-")
        for c in code:
            if c.find("a") is not None:
                all_links.append("https://fr.whoscored.com" + c.find("a").get("href"))
            
    finally:
        driver.quit()

    return all_links


In [ ]:
urls = ["https://fr.whoscored.com/regions/206/tournaments/4/seasons/10317/stages/23401/show/espagne-laliga-2024-2025",
        "https://fr.whoscored.com/regions/108/tournaments/5/seasons/10375/stages/23490/show/italie-serie-a-2024-2025",
        "https://fr.whoscored.com/regions/81/tournaments/3/seasons/10365/stages/23471/show/allemagne-bundesliga-2024-2025",
        "https://fr.whoscored.com/regions/74/tournaments/22/seasons/10329/stages/23414/show/france-ligue-1-2024-2025",
        "https://fr.whoscored.com/regions/252/tournaments/2/seasons/10316/stages/23400/show/angleterre-premier-league-2024-2025"]

In [165]:
url = "https://fr.whoscored.com/regions/74/tournaments/22/seasons/10329/stages/23414/fixtures/france-ligue-1-2024-2025"
links = get_link_current_month(url)
links

['https://fr.whoscored.com/matches/1824923/preview/france-ligue-1-2024-2025-nice-nantes']

# Rate from Whoscored

In [88]:
import re
import pandas as pd
from tqdm import tqdm
import ace_tools_open as tools
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from datetime import datetime

def get_whoscored_links(league):
    df = pd.read_csv("Whoscored/urls/clean/links.csv")
    links = []
    for link in df[df["ligue"] == league].iloc[:, 0].tolist():
        links.append("https://fr.whoscored.com/" + link)
    return links


def get_event(soup):
    code = soup.find_all("a", class_="team-link")
    team_dom = code[0].text
    team_ext = code[1].text

    c = soup.find_all("div", class_="info-block cleared")[2].text.replace("Coup d'envoi:", "").split(", ")
    date_str = c[1]
    date_obj = datetime.strptime(date_str, '%d-%b-%y')  # Convertir en objet datetime
    formatted_date = date_obj.strftime('%d/%m/%Y')  # Reformater en "01/03/2025"
    date = formatted_date
    heure = c[0].split("Date")[0]
    stade = soup.find_all("span", class_="value")[0].text
    #arbitre = soup.find_all("span", class_="value")[2].text.split(". ")[1]
    affluence = int(soup.find_all("span", class_="value")[1].text.replace(",", ""))

    return date, heure, team_dom, team_ext, affluence, stade


def get_rate(soup, link):

    # Extraction des notes
    rate = [c.text.strip() if c.text.strip() else "NaN" for c in soup.find_all("span", class_="player-stat-value")]

    # Extraction des noms des joueurs
    player = [c.text.strip() for c in soup.find_all("span", class_="player-name")][:len(rate)]
    n = len(player)

    # Event
    match = re.search(r'/live/[^/-]+-([^/]+)-(\d{4}-\d{4})', link)
    championnat = [match.group(1).replace('-', ' ').title()] * n
    saison = [match.group(2)] * n

    date, heure, team_dom, team_ext, affluence, stade = get_event(soup)

    date = [date] * n
    heure = [heure] * n
    team_dom = [team_dom] * n
    team_ext = [team_ext] * n
    affluence = [affluence] * n
    stade = [stade] * n

    # Création du DataFrame
    df_rate = pd.DataFrame({'championnat':championnat, 'saison': saison, 'date': date, 'heure': heure, 'stade':stade, 'Equipe1': team_dom, 'Equipe2': team_ext, 'affluence':affluence, 'joueur': player, 'rate': rate})

    return df_rate


def get_page_rate(links):
    options = Options()
    options.add_argument("--headless")  # Mode headless activé
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")

    service = Service(GeckoDriverManager().install())
    driver = webdriver.Firefox(service=service, options=options)
    driver.set_page_load_timeout(30)
    
    rates = pd.DataFrame()
    errors = []

    try:
        for link in tqdm(links):
            try:
                driver.get(link)
                WebDriverWait(driver, 15).until(
                    lambda d: any(e.text.strip() for e in d.find_elements(By.CLASS_NAME, "player-stat-value"))
                )
                html = driver.page_source
                soup = BeautifulSoup(html, "html.parser")

                rates = pd.concat([rates, get_rate(soup, link)])
            except Exception as e:
                print(f"❌ Erreur : {e}")
                errors.append(link)
        
    finally:
        driver.quit()

    return rates, pd.DataFrame(errors)


In [89]:
links = get_whoscored_links("ligue-1")
rates, errors = get_page_rate(links)
tools.display_dataframe_to_user("Rates :", rates)
try:
    tools.display_dataframe_to_user("Errors :", errors)
except:
    tools.display_dataframe_to_user("Errors :", pd.DataFrame())

 92%|█████████▏| 220/239 [05:47<00:40,  2.13s/it]

❌ Erreur : Message: out of memory
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:199:5
UnknownError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:910:5
wrap@chrome://remote/content/shared/webdriver/Errors.sys.mjs:145:12
sendError@chrome://remote/content/marionette/message.sys.mjs:284:29



 92%|█████████▏| 221/239 [06:17<03:09, 10.53s/it]

❌ Erreur : Message: Navigation timed out after 30000 ms
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:199:5
TimeoutError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:812:5
bail@chrome://remote/content/marionette/sync.sys.mjs:197:19



100%|██████████| 239/239 [06:47<00:00,  1.71s/it]


Rates :


Errors :


0
Loading ITables v2.2.5 from the internet... (need help?)
